# Scraping 10-Ks and 10-Qs for Alpha (Data Cleaning)
## THESIS:
Major text changes in 10-K and 10-Q filings over time indicate significant decreases in future returns. We find alpha in shorting the companies with the largest text changes in their filings and buying the companies with the smallest text changes in their filings.

Lauren Cohen, Christopher Malloy and Quoc Nguyen argue in their [recent paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1658471) that we can simply analyze textual changes in 10-Ks and 10-Qs to predict companies' future stock returns. (For an overview of this paper from Lauren Cohen, see the [Lazy Prices interview](https://www.youtube.com/watch?v=g96gROyc3wE) from QuantCon 2018.)

### Introduction
Publicly listed companies in the U.S. are required by law to file "10-K" and "10-Q" reports with the [Securities and Exchange Commission](https://www.sec.gov/) (SEC). These reports provide both qualitative and quantitative descriptions of the company's performance, from revenue numbers to qualitative risk factors.

When companies file 10-Ks and 10-Qs, they are required to disclose certain pieces of information. For example, companies are required to report information about ["significant pending lawsuits or other legal proceedings"](https://www.sec.gov/fast-answers/answersreada10khtm.html). As such, 10-Ks and 10-Qs often hold valuable insights into a company's performance.

These insights can be difficult to access. The average 10-K was [42,000 words long](https://www.wsj.com/articles/the-109-894-word-annual-report-1433203762) in 2013. Sheer length, dense language and lots of boilerplate can further obfuscate true meaning for many investors.

### Hypothesis

Companies make major textual changes to their 10-Ks when major things happen to their business. Textual changes to 10-Ks are a signal of future stock price movement.

Since the vast majority (86%) of textual changes have negative sentiment, we expect that major textual changes signal a decrease in stock price (Cohen et al. 2018).

Thus, we expect to find alpha by shorting companies with large textual changes in their 10-Ks.

### Implementation

1. Scrape every publicly traded company's 10-Ks from the SEC EDGAR database. Remove extraneous content from the 10-Ks(numerical tables, HTML tags, XBRL tags, etc).
2. For each company, compute cosine similarity and Jaccard similarity scores over the sequence of its 10-Ks. Each 10-K is compared to the previous year's 10-K.
3. Compile these scores into one dataset.

In [1]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm

## 1. Data Scraping

In [5]:
# Download the tickers manually from:
# Nasdaq: https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download
# AMEX: https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=amex&render=download
# NYSE: https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download

# Get lists of tickers for NASDAQ, NYSE, AMEX
nasdaq_tickers = pd.read_csv('tickers/nasdaq_tickers.csv')
nyse_tickers = pd.read_csv('tickers/nyse_tickers.csv')
amex_tickers = pd.read_csv('tickers/amex_tickers.csv')
print('loaded tickers')

nasdaq_tickers.drop(labels='Unnamed: 8', axis='columns', inplace=True)
print('dropped unnamed nasdaq_tickers')
nyse_tickers.drop(labels='Unnamed: 8', axis='columns', inplace=True)
print('dropped unnamed nyse_tickers')
amex_tickers.drop(labels='Unnamed: 8', axis='columns', inplace=True)
print('dropped unnamed amex_tickers')

# Create full list of tickers/names
tickers = list(set(list(nasdaq_tickers['Symbol']) + list(nyse_tickers['Symbol']) + list(amex_tickers['Symbol'])))
print('done!')

loaded tickers
dropped unnamed nasdaq_tickers
dropped unnamed nyse_tickers
dropped unnamed amex_tickers
done!


The SEC indexes company filings by its own internal identifier, the "Central Index Key" (CIK). We'll need to translate tickers into CIKs in order to search for company filings on EDGAR.

(The code below is an edited version of [this gist](https://gist.github.com/dougvk/8499335).)

In [6]:
def MapTickerToCik(tickers):
    url = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    cik_re = re.compile(r'.*CIK=(\d{10}).*')

    cik_dict = {}
    for ticker in tqdm(tickers): # Use tqdm lib for progress bar
        results = cik_re.findall(requests.get(url.format(ticker)).text)
        if len(results):
            cik_dict[str(ticker).lower()] = str(results[0])
    
    return cik_dict

In [7]:
cik_dict = MapTickerToCik(tickers)

100%|██████████| 7340/7340 [1:51:36<00:00,  1.10it/s]  


In [8]:
# Clean up the ticker-CIK mapping as a DataFrame
ticker_cik_df = pd.DataFrame.from_dict(data=cik_dict, orient='index')
ticker_cik_df.reset_index(inplace=True)
ticker_cik_df.columns = ['ticker', 'cik']
ticker_cik_df['cik'] = [str(cik) for cik in ticker_cik_df['cik']]

Our ultimate goal is to link each ticker to a unique CIK.

However, some CIKs might be linked to multiple tickers. For example, different [share classes](https://www.investopedia.com/terms/s/share_class.asp) within the same company would all be linked to the same CIK. Let's get rid of these duplicate mappings.

In [9]:
# Check for duplicated tickers/CIKs
print("Number of ticker-cik pairings:", len(ticker_cik_df))
print("Number of unique tickers:", len(set(ticker_cik_df['ticker'])))
print("Number of unique CIKs:", len(set(ticker_cik_df['cik'])))

Number of ticker-cik pairings: 5950
Number of unique tickers: 5950
Number of unique CIKs: 5478


It looks like about 200 (4.5%) CIKs are linked to multiple tickers. To eliminate the duplicate mappings, we'll simply keep the ticker that comes first in the alphabet. In most cases, this means we'll keep the class A shares of the stock.

In [11]:
ticker_cik_df = ticker_cik_df.sort_values(by='ticker')
ticker_cik_df.drop_duplicates(subset='cik', keep='first', inplace=True)
# Check that we've eliminated duplicate tickers/CIKs
print("Number of ticker-cik pairings:", len(ticker_cik_df))
print("Number of unique tickers:", len(set(ticker_cik_df['ticker'])))
print("Number of unique CIKs:", len(set(ticker_cik_df['cik'])))

Number of ticker-cik pairings: 5478
Number of unique tickers: 5478
Number of unique CIKs: 5478


At this point, we have a list of the CIKs for which we want to obtain 10-Ks and can now begin scraping from EDGAR.

As with many web scraping projects, we'll need to keep some technical considerations in mind:

- We're scraping a lot of data, so it's unlikely that we'll be able to do it all in one session without something breaking. As such, we should make sure that our scraper can easily pick up where it left off without having to re-scrape anything.
- We probably want to log warnings/errors and save that log, just in case we need to reference it later.
- The SEC limits users to [10 requests per second](https://www.sec.gov/developer)

In [2]:
# This is a checkpoint since it took over 2 hours to generate
try:
    ticker_cik_df.to_csv('tickers/tickers_and_ciks.csv')
except:
    ticker_cik_df = pd.read_csv('tickers/tickers_and_ciks.csv')

In [3]:
def WriteLogFile(log_file_name, text):
    
    '''
    Log all notes and errors from scraping sessio
    
    Parameters
    ----------
    log_file_name : str
        Name of the log file (should be a .txt file).
    text : str
        Text to write to the log file.
    Returns
    -------
    None.
    '''
    
    with open(log_file_name, "a") as log_file:
        log_file.write(text)
    return

The function below scrapes all 10-Ks for one particular CIK. Our web scraper primarily depends on the requests and BeautifulSoup libraries.

Note that the scraper creates a different directory for each CIK, and puts all the filings for that CIK within that directory. After scraping, your file structure should look like this:
```
- 10Ks
    - CIK1
        - 10K #1
        - 10K #2
        ...
    - CIK2
        - 10K #1
        - 10K #2
        ...
    - CIK3
        - 10K #1
        - 10K #2
        ...  
    ...
```    
The scraper will create the directory for each CIK. However, we need to create different directories to hold our 10-K files. The exact pathname depends on your local setup, so you'll need to fill in the correct pathname_10k.

In [18]:
def Scrape10K(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    '''
    Scrapes all 10-Ks and 10-K405s for a CIK from EDGAR.
    
    Parameters
    ----------
    browse_url_base : str
        Base URL for browsing EDGAR.
    filing_url_base : str
        Base URL for filings listings on EDGAR.
    doc_url_base : str
        Base URL for one filing's document tables
        page on EDGAR.
    cik : str
        Central Index Key.
    log_file_name : str
        Name of the log file (should be a .txt file).
    Returns
    -------
    None.
   
    '''
    cik = str(cik)
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    os.chdir(str(cik))
    
    print('Scraping CIK', cik)
    
    # Request list of 10-K filings
    res = requests.get(browse_url_base % cik)
    
    # If request failed, log failure and exit
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return
    
    soup = bs.BeautifulSoup(res.text, "lxml")
    html_tables = soup.find_all('table')
    
    # Check if table exists
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    # Get 10-K and 10-K405 filings
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]
    filings_table = filings_table[(filings_table['Filings'] == '10-K') | (filings_table['Filings'] == '10-K405')]

    # If filings table doesn't have any entries...
    if len(filings_table)==0:
        os.chdir('..')
        return
    
    # Get access number for each 10-K and 10-K405
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    # scrape each document
    for index, row in filings_table.iterrows():
        
        # Get access number for filing
        acc_no = str(row['Acc_No'])
        
        # Navigate to page for filing
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        
        # If request fails, log failure
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
        # Parse table of documents for filing
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        # Get 10-K and 10-K405 entries for the filing
        docs_table = docs_table[(docs_table['Type'] == '10-K') | (docs_table['Type'] == '10-K405')]
        if len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        docname = docs_table['Document']
        
        # If first entry unavailable log failure
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue     
        
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname))
        
        # If request fails, log failure
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue

        # Save
        if '.txt' in docname:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    os.chdir('..')
        
    return

Now that we've defined our scraper functions, let's scrape.

(A note from the future: we're scraping a lot of data, which takes time and space. For reference, these functions ultimately scraped 125 GB of 10-Ks; the scraping took roughly 20 hours total.)

In [19]:
os.chdir('..')

In [20]:
files = [f for f in os.listdir() if os.path.isfile(f)]
files

['notebook_473_zipline.ipynb',
 '.DS_Store',
 'quantopian_notebook_474.html',
 'notebook_474_zipline.ipynb',
 'quantopian_notebook_473.html',
 'Scraping 10-Ks and 10-Qs for Alpha.html',
 'notebook_473_transcribed.ipynb',
 'Analyzing Alpha in 10-Ks and 10-Qs.html',
 'notebook_474_transcribed.ipynb']

In [21]:
# Scrape 10-Ks

browse_url_base_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

pathname_10k = '10ks'
os.chdir(pathname_10k)

# (log file name = the time we initiate scraping session)
time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
with open(log_file_name, 'a') as log_file:
    log_file.close()

# Iterate over CIKs and scrape 10-Ks
for cik in tqdm(ticker_cik_df['cik']):
    Scrape10K(browse_url_base=browse_url_base_10k, 
          filing_url_base=filing_url_base_10k, 
          doc_url_base=doc_url_base_10k, 
          cik=cik,
          log_file_name=log_file_name)

  0%|          | 0/5478 [00:00<?, ?it/s]

Already scraped CIK 1090872
Already scraped CIK 1675149
Scraping CIK 1420529


  0%|          | 3/5478 [00:00<09:09,  9.97it/s]

Scraping CIK 1802457


  0%|          | 4/5478 [00:00<14:17,  6.38it/s]

...

100%|█████████▉| 5476/5478 [13:55:48<00:09,  4.94s/it]

Scraping CIK 1621443


100%|█████████▉| 5477/5478 [13:55:53<00:04,  4.88s/it]

Scraping CIK 846475


100%|██████████| 5478/5478 [13:56:02<00:00,  9.16s/it]


We now have 10-Ks in HTML or plaintext (.txt) format for each CIK. Before computing our similarity scores we need to clean the files.

As outlined in the paper, we will:

> ... remove all tables (if their numeric character content is greater than 15%), HTML tags, XBRL tables, exhibits, ASCII-encoded PDFs, graphics, XLS, and other binary files.

In [27]:
def RemoveNumericalTables(soup):
    
    '''
    Removes tables with >15% numerical characters.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup
        with numerical tables removed.
    '''
    
    # Determine percentage of numerical characters in a table
    def GetDigitPercentage(tablestring):
        if len(tablestring)>0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers/length
        else:
            return 1
    
    # Evaluate numerical character % for each table and remove if > 15%
    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text())>0.15]
    
    return soup

In [28]:
def RemoveTags(soup):
    
    '''
    Drops HTML tags, newlines and unicode text from
    filing text.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    text : str
        Filing text. 
    '''
    
    # Remove HTML tags with get_text
    text = soup.get_text()
    
    # Remove newline characters
    text = text.replace('\n', ' ')
    
    # Replace unicode characters with their
    # "normal" representations
    text = unicodedata.normalize('NFKD', text)
    
    return text

In [35]:
def ConvertHTML(cik):
    
    '''
    Removes numerical tables, HTML tags,
    newlines, unicode text, and XBRL tables.
    
    Parameters
    ----------
    cik : str
        Central Index Key used to scrape files.
    
    Returns
    -------
    None.
    '''
    
    cik = str(cik)
    try: 
        os.chdir(cik)
    except FileNotFoundError:
        print("Could not find directory for CIK", cik)
        return
        
    print("Parsing CIK %s..." % cik)
    parsed = False
    
    try:
        os.mkdir('rawtext')
    # If exists, continue
    # Can't exit at this point because we might be
    # partially through parsing text files, so we need to continue
    except OSError:
        pass
    
    # Get list of scraped files
    file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname))]
    
    for filename in file_list:
            
        # Check if file has already been cleaned
        new_filename = filename.replace('.html', '.txt')
        text_file_list = os.listdir('rawtext')
        if new_filename in text_file_list:
            continue
        
        # Clean file
        with open(filename, 'r') as file:
            parsed = True
            soup = bs.BeautifulSoup(file.read(), "lxml")
            soup = RemoveNumericalTables(soup)
            text = RemoveTags(soup)
            with open('rawtext/'+new_filename, 'w') as newfile:
                newfile.write(text)
    
    if parsed==False:
        print("Already parsed CIK", cik)
    
    os.chdir('..')
    return

We can now apply this function to each of our 10-K files.

In [37]:
os.chdir(pathname_10k)

# Iterate over CIKs and clean HTML filings
for cik in tqdm(ticker_cik_df['cik']):
    ConvertHTML(cik)

  0%|          | 0/5478 [00:00<?, ?it/s]

Parsing CIK 1090872...
Already parsed CIK 1090872
Parsing CIK 1675149...
Already parsed CIK 1675149
Parsing CIK 1420529...
Already parsed CIK 1420529
Parsing CIK 1802457...
Already parsed CIK 1802457
Parsing CIK 1209028...


  0%|          | 5/5478 [00:45<13:49:43,  9.10s/it]

Parsing CIK 6201...


  0%|          | 6/5478 [01:45<37:16:03, 24.52s/it]

...

100%|█████████▉| 5476/5478 [20:47:54<00:11,  5.69s/it]

Parsing CIK 1621443...


100%|█████████▉| 5477/5478 [20:48:00<00:05,  5.70s/it]

Parsing CIK 846475...


100%|██████████| 5478/5478 [20:48:08<00:00, 13.67s/it]


After running the two cells above, we have cleaned plaintext 10-K filings for each CIK. At this point, our file structure looks like this:
```
- 10Ks
    - CIK1
        - 10K #1
        - 10K #2
        ...
        - rawtext
    - CIK2
        - 10K #1
        - 10K #2
        ...
        - rawtext
    - CIK3
        - 10K #1
        - 10K #2
        ...
        - rawtext
    ...
```
We can now begin computing our alpha factor (similarity scores).

## 2. Computing Similarity Scores
We'll use [cosine similarity](http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity) and [Jaccard similarity](http://scikit-learn.org/stable/modules/model_evaluation.html#jaccard-similarity-score) to compare documents.

(The original paper also uses two other, simpler similarity measures, but cosine and Jaccard appeared to result in the best alpha factor performance -- and are much less computationally intensive to compute.)

In [38]:
def ComputeCosineSimilarity(words_A, words_B):
    
    '''
    Compute cosine similarity between document A and
    document B.
    
    Parameters
    ----------
    words_A : set
        Words in document A.
    words_B : set
        Words in document B
        
    Returns
    -------
    cosine_score : float
        Cosine similarity between document
        A and document B.  
    '''
    
    # Complete set of words
    words = list(words_A.union(words_B))
    
    # Words in A
    vector_A = [1 if x in words_A else 0 for x in words]
    
    # Words in B
    vector_B = [1 if x in words_B else 0 for x in words]
    
    # Compute cosine score using scikit-learn
    array_A = np.array(vector_A).reshape(1, -1)
    array_B = np.array(vector_B).reshape(1, -1)
    cosine_score = cosine_similarity(array_A, array_B)[0,0]
    
    return cosine_score

In [39]:
def ComputeJaccardSimilarity(words_A, words_B):
    
    '''
    Compute Jaccard similarity between document A and
    document B.
    
    Parameters
    ----------
    words_A : set
        Words in document A.
    words_B : set
        Words in document B
    Returns
    -------
    jaccard_score : float
        Jaccard similarity between document
        A and document B.
    '''
    
    # Count number of words in both A and B
    words_intersect = len(words_A.intersection(words_B))
    
    # Count number of words in A or B
    words_union = len(words_A.union(words_B))
    
    # Compute Jaccard similarity score
    jaccard_score = words_intersect / words_union
    
    return jaccard_score

We don't mind much if we're comparing a 10-K from 2006 to a 10-K from 2002. This is because we don't have to worry about non-substantive quarter-on-quarter differences as we would with 10-Qs. In fact, it might actually be better if our data reflects textual changes in 10-Ks across "absent" years.

In [46]:
def ComputeSimilarityScores10K(cik):
    
    '''
    Computes cosine and Jaccard similarity scores
    over 10-Ks for a particular CIK.
    
    Parameters
    ----------
    cik: str
        Central Index Key used to scrape and name
        files.
        
    Returns
    -------
    None.
    '''
    
    cik = str(cik)
    # Open the directory that holds plaintext
    # filings for the CIK
    os.chdir(cik+'/rawtext')
    print("Parsing CIK %s..." % cik)
    
    # Get list of files to compute scores
    file_list = [fname for fname in os.listdir() if not 
                 (fname.startswith('.') | os.path.isdir(fname))]
    file_list.sort()
    
    # Check if scores have already been calculated...
    try:
        os.mkdir('../metrics')
    except OSError:
        print("Already parsed CIK %s..." % cik)
        os.chdir('../..')
        return
    
    # Check if enough files exist to compute sim scores...
    if len(file_list) < 2:
        print("No files to compare for CIK", cik)
        os.chdir('../..')
        return
    
    # Initialize dataframe to store sim scores
    dates = [x[-14:-4] for x in file_list]
    cosine_score = [0]*len(dates)
    jaccard_score = [0]*len(dates)
    data = pd.DataFrame(columns={'cosine_score': cosine_score, 
                                 'jaccard_score': jaccard_score},
                       index=dates)
        
    # Open first file
    file_name_A = file_list[0]
    with open(file_name_A, 'r') as file:
        file_text_A = file.read()
        
    # Iterate over each 10-K file...
    for i in range(1, len(file_list)):

        file_name_B = file_list[i]

        # Get file text B
        with open(file_name_B, 'r') as file:
            file_text_B = file.read()

        words_A = set(re.findall(r"[\w']+", file_text_A))
        words_B = set(re.findall(r"[\w']+", file_text_B))
        cosine_score = ComputeCosineSimilarity(words_A, words_B)
        jaccard_score = ComputeJaccardSimilarity(words_A, words_B)

        # Store score values
        date_B = file_name_B[-14:-4]
        data.at[date_B, 'cosine_score'] = cosine_score
        data.at[date_B, 'jaccard_score'] = jaccard_score

        # set prev doc to current doc for next iteration
        file_text_A = file_text_B

    # Save scores
    os.chdir('../metrics')
    data.to_csv(cik+'_sim_scores.csv', index=False)
    os.chdir('../..')

The values correspond with the date of the later document (document B). This is because values (similarity scores) depend on two things: the content of document A, and the content of document B. A is the earlier document. Since A precedes B, we wouldn't know about the factor values at date_A; the content of B would not yet be available. However, we would know about the factor values at date_B -- at that point, both the content of A and the content of B would be available. As such, we'll store our values according to date_B.

Let's go ahead and apply these functions to our stored 10-Ks.

In [47]:
for cik in tqdm(ticker_cik_df['cik']):
    ComputeSimilarityScores10K(cik)

  0%|          | 0/5478 [00:00<?, ?it/s]

Parsing CIK 1090872...
No files to compare for CIK 1090872
Parsing CIK 1675149...
No files to compare for CIK 1675149
Parsing CIK 1420529...
No files to compare for CIK 1420529
Parsing CIK 1802457...
No files to compare for CIK 1802457
Parsing CIK 1209028...


  0%|          | 5/5478 [00:02<52:58,  1.72it/s]

Parsing CIK 6201...


  0%|          | 6/5478 [00:06<2:14:42,  1.48s/it]

...

100%|█████████▉| 5476/5478 [1:44:13<00:01,  1.63it/s]

Parsing CIK 1621443...


100%|█████████▉| 5477/5478 [1:44:14<00:00,  1.59it/s]

Parsing CIK 846475...


100%|██████████| 5478/5478 [1:44:15<00:00,  1.14s/it]


After computing the similarity scores, our file structure looks like this:
```
- 10Ks
    - CIK1
        - 10K #1
        - 10K #2
        ...
        - rawtext
        - metrics
    - CIK2
        - 10K #1
        - 10K #2
        ...
        - rawtext
        - metrics
    - CIK3
        - 10K #1
        - 10K #2
        ...
        - rawtext
        - metrics
    ...
```
The similarity scores for each CIK are stored in the metrics directory as a .csv file.